In [43]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix_two/dw-predict-car/"

/content/drive/My Drive/Colab Notebooks/dw_matrix_two/dw-predict-car


In [0]:
# !pip install --upgrade tables
# !pip install eli5

In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import  mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [84]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [45]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [54]:
X = df[ ['car_id'] ].values
y = df['car_id'].values

model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)

mean_absolute_error(y, y_pred)

40104.125306754606

In [55]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [60]:
df['price_currency'].value_counts(normalize=True) * 100

PLN    100.0
Name: price_currency, dtype: float64

In [0]:
df = df[df['price_currency'] != 'EUR']

## Features

In [0]:
SUFFIX_CAT='__cat'

for feat in df.columns.values:
  if isinstance(df[feat][0], list): continue
  
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat: continue
  #   df[feat] = factorized_values
  # else:
  df[feat+SUFFIX_CAT] = factorized_values

In [82]:
df.shape

(106290, 309)

In [87]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [88]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19695.13091100928

In [98]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

imp = PermutationImportance(m, random_state=5).fit(X,y)
eli5.show_weights(imp, features_names=cat_feats)

TypeError: ignored